In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

from selenium import webdriver

from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import scrapy
import json
from scrapy.http import TextResponse

import time

In [2]:
today = pd.to_datetime('today').date()

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
df_properties = []
try:
    for i in range(1, 53):
        base = 'https://habi.co/venta-apartamentos'
        end = '?page=' + f'{i}'
        URL = 'https://habi.co/venta-apartamentos' + end
        print('base URL = ', URL)

        driver.get(URL)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        links = soup.find_all('a', {'class': 'card-details'})
        start_url = [l.get('href') for l in links]
        print('# of cards in this page', len(start_url))

        for i in range(len(start_url)):
            base = 'https://habi.co/'
            index = start_url[i].find('venta-apartamento')
            middle = start_url[i][index:]
            end = '/page-data.json'
            url = base + 'page-data/' + middle + end

            try:
                response = requests.get(url)
                response.raise_for_status()

                raw_data = response.json()
                description = raw_data['result']['pageContext']['description']
                row_description = pd.DataFrame([description])
                row_description = row_description.rename(columns={'0': 'description'})
                detalles_propiedad = raw_data['result']['pageContext']['propertyDetail']['property']['detalles_propiedad']
                row_details = pd.DataFrame([detalles_propiedad])
                row_property = pd.concat([row_description, row_details], axis=1)
                df_properties.append(row_property)
            except Exception as e:
                print(f"An error occurred for URL {url}: {str(e)}")

except Exception as e:
    print(f"An error occurred during the scraping process: {str(e)}")
driver.quit()

if df_properties:
    final_df = pd.concat(df_properties, ignore_index=True)
    final_df.to_csv(f'final_df_{today}.csv', index=False)
else:
    print("No data to save.")

base URL =  https://habi.co/venta-apartamentos?page=1
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=2
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=3
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=4
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=5
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=6
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=7
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=8
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=9
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=10
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=11
# of cards in this page 32
base URL =  https://habi.co/venta-apartamentos?page=12
# of cards in this page 32
base URL =  https://habi.

In [5]:
final_df

,0,area,tipo_inmueble,estrato,conjunto,direccion,precio_venta,precio_anterior,discount_rate,barriocomun,...,garajes,telefono,anos_antiguedad,last_admin_price,is_private,zona_metropolitana,contacto_zona,zona_mediana,latitud,longitud
0,"Casa en venta en venta de 95m2, con vista exte...",95.0,Casa,4,Casa en Venta,Carrera 95A # 48A-63,405000000,417000000,3,Valle de Lili,...,1,+57 3502989928,11,0,0,Cali,+57 3145773391,Comuna 17,3.373470,-76.514786
1,"Apartamento en venta de 60 m2, con vista inter...",60.0,Apartamento,3,Edificio Parque de Villa Javier,Carrera 6 #8A-35 SUR,280000000,291000000,4,San Javier,...,1,+57 3057060019,25,218911,1,Bogotá,+57 3145773391,SOSIEGO,4.580091,-74.085320
2,"Apartamento en venta de 49m2, con vista exteri...",49.0,Apartamento,2,Bosques de Bogota 2,Carrera 14i # 136a-90sur,95000000,105000000,10,Usme Centro,...,0,+57 3107909292,10,60000,0,Bogotá,+57 3145773391,CIUDAD USME,4.475051,-74.126536
3,"Casa en venta de 91m2, con vista exterior, par...",91.0,Casa,5,Casa,Carrera 101 # 45 - 63,400000000,410000000,2,Valle del Lili,...,1,+57 3156341453,10,0,0,Cali,+57 3145773391,Comuna 17,3.366014,-76.518327
4,"Casa en venta de 170m2, con vista interior, (I...",170.0,casa con conjunto cerrado,4,La Coruña,Carrera 17 # 146 - 62,685000000,695000000,1,Cedritoa,...,2,+57 3132619026,32,500000,1,Bogotá,+57 3145773391,LOS CEDROS,4.728894,-74.043100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,"Apartamento en venta de 75 m2, con vista inter...",75.0,Apartamento,5,Edificio Cerezo,carrera 48 # 125-21,406700000,None,None,El Batan,...,2,+57 3003572445,15,404000,1,Bogotá,+57 3145773391,LA ALHAMBRA,4.706093,-74.057369
1627,"Apartamento en venta de 103m2, con vista inter...",103.0,Apartamento,5,Saint Michel,Calle 104b # 57-23,446000000,None,None,Puente Largo,...,1,0,27,0,0,Bogotá,+57 3145773391,LA ALHAMBRA,4.692691,-74.065108
1628,"Apartamento en venta de 47m2, con vista exteri...",56.0,Apartamento,2,San Bernandino 1,Calle 72b sur # 89a-01,121100000,None,None,Villas Del Progreso,...,0,+57 3023192270,14,0,1,Bogotá,+57 3145773391,BOSA OCCIDENTAL,4.624487,-74.201517
1629,"Apartamento en venta de 44m2, con vista interi...",47.0,Apartamento,1,Guayacán Parques de Bogotá,Calle 88 sur # 88C-90,124000000,None,None,San Bernardino,...,0,+57 3246814262,6,0,1,Bogotá,+57 3145773391,TINTAL SUR,4.621051,-74.211668
